# LHC Machine Learing workshop challenge
https://gitlab.cern.ch/IML-WG/IML_challenge_2018/wikis/home

Task: Regress the soft-drop mass of jets with high transverse momentum. Jets are complex physical objects, often containing a spray of particles.https://en.wikipedia.org/wiki/Jet_(particle_physics)

STEP III

Reload the data here. Use the constituents put through the trained encoder, the usse the code concatenated along with reconstruted jet quantities as input feature vectors

In [1]:
#get the data
import numpy as np

my_array=np.load('../data/qcd.npy',encoding='bytes')

my_rec_array=my_array.view(np.recarray)


In [3]:
fields=my_array.dtype.names

In [5]:
#del my_train_val_array
num_jets=my_rec_array.shape[0]
for field in fields:
    print(field)
    x=getattr(my_rec_array,field)
    x=np.reshape(x,[1,num_jets])
    try:
        my_train_val_array=np.concatenate((my_train_val_array,x))
    except:
        print(field)
        my_train_val_array=x
        

genjet_sd_m
genjet_sd_m
recojet_pt
recojet_eta
recojet_phi
recojet_m
recojet_sd_pt
recojet_sd_eta
recojet_sd_phi
recojet_sd_m
n_constituents
constituents_pt
constituents_eta
constituents_phi
constituents_charge
constituents_dxy
constituents_dz
constituents_Eem
constituents_Ehad


In [6]:
my_train_val_array=my_train_val_array.T
my_train_val_array.shape

(1042167, 18)

In [7]:
from sklearn.model_selection import train_test_split

train_array, val_array = train_test_split(my_train_val_array, test_size=0.30,random_state=2158)
train_array.shape,val_array.shape

((729516, 18), (312651, 18))

In [8]:
reco_features_train=train_array[:,0:10] #recofeatures+target
#features1[:3]
const_features_train=train_array[:,9:] #constituents
reco_features_val=val_array[:,0:10] #recofeatures+target
#features1[:3]
const_features_val=val_array[:,9:] #constituents

In [1]:
from keras import regularizers,optimizers
from keras.layers import Reshape,Flatten,Input, Dense,Conv2D, MaxPooling2D, UpSampling2D,ELU,LeakyReLU,Dropout,BatchNormalization,concatenate
from keras.models import Model,load_model
from keras import backend as K
from keras import callbacks


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Get training constituents

In [49]:

train_consts=np.load('train_consts.npz')
train_consts.shape

(3000, 1, 8, 130)

Get validation constituents

In [51]:
val_consts=np.load('train_consts.npz')
val_consts.shape

(3000, 1, 8, 130)

Load Encoder

In [62]:
encoder=load_model('my_encoder.h5')
train_features=encoder.predict(train_consts)


[[ -1.38968982e+03  -1.88133179e+03   4.27937256e+02 ...,  -1.11373009e+02
   -1.02983374e+03  -3.97367603e+03]
 [ -1.84785430e+04  -2.05137285e+04  -1.63939961e+04 ...,  -6.97378174e+03
   -3.32072695e+04  -4.56478086e+04]
 [ -9.36249316e+03  -1.19950879e+04  -5.68674902e+03 ...,  -2.88605396e+03
   -2.03736152e+04  -2.75708340e+04]
 ..., 
 [  1.19594536e+02   1.40916300e+00  -3.01769348e+02 ...,  -5.73151306e+02
    3.45790314e+02   5.53951035e+01]
 [ -1.94455352e+04  -5.01653398e+04   2.11435234e+04 ...,   8.77271680e+03
   -3.32262031e+05  -1.91350312e+05]
 [ -4.48036133e+03  -5.78808655e+02  -7.42071826e+03 ...,  -1.36916467e+03
    5.13298767e+02  -2.85274231e+02]]


In [69]:
train_target=reco_features_train[0:3000,0]
#train_target=np.reshape(train_target,[tr])
print(train_target.shape)
reco_features_trial=reco_features_train[0:3000,1:]

(3000,)


Stack together code and code from encoder

In [ ]:
train_final_trial=np.hstack((reco_features_trial,train_features))
#train_final_trial=np.reshape(train_final_trial,[train_final_trial.shape[0],1,train_final_trial.shape[1]])
train_final_trial.shape

In [2]:
#regressor model
input_vec=Input(shape=(59,))

x=BatchNormalization(axis=1)(input_vec)
x=Dense(400,name='densea')(x)
x=Dropout(0.4)(x)
x=LeakyReLU(alpha=0.3)(x)
x=BatchNormalization(axis=1)(x)

x=Dense(300,name='denseb')(x)
x=Dropout(0.4)(x)
x=LeakyReLU(alpha=0.3)(x)
x=BatchNormalization(axis=1)(x)

x=Dense(150,name='densec')(x)
x=Dropout(0.3)(x)
x=LeakyReLU(alpha=0.3)(x)
x=BatchNormalization(axis=1)(x)

x=Dense(50,name='densed',activation='elu')(x)
#x=BatchNormalization(axis=1)(x)
x=Dropout(0.1)(x)
#x=LeakyReLU(alpha=0.0001)(x)

y_pred=Dense(1,name='denseE',activation='linear')(x)

regressor=Model(input_vec,y_pred)
regressor.compile(optimizer='adadelta', loss='MSE')
regressor.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 59)                0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 59)                236       
_________________________________________________________________
densea (Dense)               (None, 400)               24000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 400)               0         
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 400)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 400)               1600      
_________________________________________________________________
denseb (Dense)               (None, 300)               120300    
__________

In [106]:
for _ in range(2):#trial, training on gpu-->see train_regressor*.py
    sd=regressor.fit(x=train_final_trial, y=train_target, epochs=1,batch_size=32,
                validation_data=[train_final_trial,train_target],
                verbose=1,
                )

Train on 3000 samples, validate on 3000 samples
Epoch 1/1
3000/3000 [==============================] - 7s - loss: 80773.7822 - val_loss: 16789.5136
Train on 3000 samples, validate on 3000 samples
Epoch 1/1
3000/3000 [==============================] - 2s - loss: 19394.7496 - val_loss: 20459.6737


In [107]:
regressor.save('regressor_trial.h5')

In [83]:
trained_model=load_model('regressor_trial.h5')

Fourth and final step is using the regressor to evaluate the custom metric a.k.a Resolution of the jet energy scale -->evaluation.ipynb